In [ ]:
import csv
import cassandra
import warnings
import datetime
import time
import dask
import pandas as pd
import numpy as np
import dask.dataframe as da
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider1 = PlainTextAuthProvider(username='orderhub', password='/1234*')
archival_cluster = Cluster(['u123kot'],port = 3176,auth_provider=auth_provider1)
session = archival_cluster.connect('orders')
#warnings.filterwarnings("ignore")

files = 1
end_file = 8
days_backup=100
backup_date = 123173     #fixed
cutoff_date = (backup_date - days_backup)
directory= '/app/Python-docker/validation/'

while (files <= end_file):
        
    try:
        print("time started: "+str(time.time()))
        df_read_backup=da.read_csv(f'{directory}/partition/backup_data_{files}.csv')        
        df_read_backup['julian_date'] = df_read_backup['order_id'].astype(str).str[1:6].astype(int)
        df_read_backup['validated_list'] = cutoff_date > df_read_backup['julian_date']        
               
        order_list = list(df_read_backup.loc[df_read_backup['validated_list'] == True]['order_id'])
    
        order_str_list = ', '.join(f"'{order}'" for order in order_list)
        
        query1 = SimpleStatement("select count(1), order_id from orders.order_events where order_id in (%s) group by order_id"%order_str_list,consistency_level=ConsistencyLevel.QUORUM)
        query_result = session.execute(query1)                
        
        df_query_result = pd.DataFrame(query_result)
    
        df_query_result['order_id'] = df_query_result['order_id'].astype(np.int64)
        df_query_result['count'] = df_query_result['count'].astype(int)     
        df_combine = pd.merge(df_read_backup.compute(), df_query_result, how="left", on=['order_id'])
        
        # #write in csv validation file
        df_combine.loc[(df_combine['validated_list'] == True) & (df_combine['count'].notna())][['order_id','count']]\
            .to_csv(f'{directory}/validating/mark_validation_{files}.csv' , index = False)
            
        ##write non-validation file
        df_combine.loc[df_combine['validated_list'] == False][['order_id']]\
            .to_csv(f'{directory}/validating/Not_validated_{files}.csv' , index = False)
            
        # #write error file
        df_combine.loc[(df_combine['validated_list'] == True) & (df_combine['count'].isna())][['order_id']]\
            .to_csv(f'{directory}/errorlogfile/logfile_{files}.csv' , index = False)
           
      
        print('No of completed files:',files) 
        files += 1


    except Exception as e:
        print (e) 
        files += 1
